## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-08-23-45-31 +0000,bbc,What happens in Taiwan's Han Kuang military ex...,https://www.bbc.com/news/articles/cly21zrnk79o
1,2025-07-08-23-19-34 +0000,nyt,U.S. Botched a Deal to Swap Venezuelans in El ...,https://www.nytimes.com/2025/07/08/world/ameri...
2,2025-07-08-23-10-10 +0000,bbc,Gemma Collins weight-loss drug advert banned,https://www.bbc.com/news/articles/cvgnr0xg7rno
3,2025-07-08-23-03-35 +0000,bbc,Action needed to halt exodus of firms leaving ...,https://www.bbc.com/news/articles/cz6g85qp0p6o
4,2025-07-08-23-03-23 +0000,bbc,"'Hero' dad, twin girls and riverside campers a...",https://www.bbc.com/news/articles/c5ygl8lpyyqo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,10
43,tariffs,7
27,uk,4
196,scandal,4
59,gaza,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
68,2025-07-08-01-05-00 +0000,wsj,President Trump reignited his global trade war...,https://www.wsj.com/economy/trade/trump-tariff...,44
13,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,40
64,2025-07-08-04-00-11 +0000,nyt,Trump and Netanyahu Meet Amid Gaza Cease-Fire ...,https://www.nytimes.com/2025/07/07/us/politics...,27
14,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,27
7,2025-07-08-22-37-57 +0000,bbc,Trump and Netanyahu meet for a second time ami...,https://www.bbc.com/news/articles/ce3newwl1zeo,26


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
68,44,2025-07-08-01-05-00 +0000,wsj,President Trump reignited his global trade war...,https://www.wsj.com/economy/trade/trump-tariff...
13,23,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
54,17,2025-07-08-09-29-38 +0000,bbc,Post Office report set to lay bare human impac...,https://www.bbc.com/news/articles/c4g267xe3y6o
64,17,2025-07-08-04-00-11 +0000,nyt,Trump and Netanyahu Meet Amid Gaza Cease-Fire ...,https://www.nytimes.com/2025/07/07/us/politics...
4,15,2025-07-08-23-03-23 +0000,bbc,"'Hero' dad, twin girls and riverside campers a...",https://www.bbc.com/news/articles/c5ygl8lpyyqo
14,13,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...
17,13,2025-07-08-19-52-56 +0000,bbc,Canadian Armed Forces members plotted 'anti-go...,https://www.bbc.com/news/articles/cly8d0jxm8po
3,12,2025-07-08-23-03-35 +0000,bbc,Action needed to halt exodus of firms leaving ...,https://www.bbc.com/news/articles/cz6g85qp0p6o
67,12,2025-07-08-01-13-56 +0000,nyt,Israel’s Deadly Assault on Iran Prison Incites...,https://www.nytimes.com/2025/07/06/world/middl...
25,10,2025-07-08-16-41-29 +0000,bbc,Imposter used AI to pose as Marco Rubio and co...,https://www.bbc.com/news/articles/crrqkyyjewno


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
